In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno

sns.set(style='darkgrid')
import matplotlib.pyplot as plt

import joblib
import os

import pandas as pd
from sklearn.ensemble import RandomForestClassifier

## Loading models

In [42]:
# Paths for client models
client_model_paths = [
    "Sisira Horizontal/trial new/Fresh_models/client_1_model.pkl",
    "Sisira Horizontal/trial new/Fresh_models/client_2_model.pkl",
    "Sisira Horizontal/trial new/Fresh_models/client_3_model.pkl",
    "Sisira Horizontal/trial new/Fresh_models/client_4_model.pkl",
    "Sisira Horizontal/trial new/Fresh_models/client_5_model.pkl"
]

In [43]:
# Iterate through each client model file, load the model, and aggregate its trees
client_models = []
for client_model_path in client_model_paths:
    # Load the client's local model
    client_models.append(joblib.load(client_model_path))

len(client_models)

5

---

In [55]:
third = client_models[2]
third.feature_names_in_,global_model.feature_names_in_# = np.sort(global_model.feature_names_in_)
global_model.feature_names_in_ = third.feature_names_in_
global_model.feature_names_in_
# ?np.sort

array(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9',
       'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17',
       'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25',
       'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'PC32', 'PC33',
       'PC34', 'PC35'], dtype=object)

---

## Collect Model Parameters

### Label/Classes Collection

In [6]:
attkLbl=[]
for model in client_models:
    thisModList = model.classes_.tolist()
    attkLbl.extend(thisModList)
    
attkLbl = list(set(attkLbl))
len(attkLbl)

9

In [14]:
features = []
for model in client_models:
    l = model.feature_names_in_.tolist()
    # print(l)
    features.extend(l)

features=list(set(temp))
print(features)

['PC12', 'PC4', 'PC20', 'PC9', 'PC32', 'PC31', 'PC19', 'PC2', 'PC33', 'PC18', 'PC27', 'PC21', 'PC1', 'PC24', 'PC34', 'PC8', 'PC16', 'PC30', 'PC17', 'PC29', 'PC5', 'PC23', 'PC15', 'PC3', 'PC25', 'PC28', 'PC22', 'PC7', 'PC6', 'PC11', 'PC13', 'PC26', 'PC14', 'PC10', 'PC35']


In [20]:
estimators = []

for model in client_models:
    estimators.extend(model.estimators_)

estimators = list(set(estimators))
print("Estimators:", estimators)


Estimators: [DecisionTreeClassifier(max_features='sqrt', random_state=1324556529), DecisionTreeClassifier(max_features='sqrt', random_state=1324556529), DecisionTreeClassifier(max_features='sqrt', random_state=423734972), DecisionTreeClassifier(max_features='sqrt', random_state=782698033), DecisionTreeClassifier(max_features='sqrt', random_state=2126768636), DecisionTreeClassifier(max_features='sqrt', random_state=249467210), DecisionTreeClassifier(max_features='sqrt', random_state=1972458954), DecisionTreeClassifier(max_features='sqrt', random_state=1111451555), DecisionTreeClassifier(max_features='sqrt', random_state=1572714583), DecisionTreeClassifier(max_features='sqrt', random_state=698027879), DecisionTreeClassifier(max_features='sqrt', random_state=1972458954), DecisionTreeClassifier(max_features='sqrt', random_state=249467210), DecisionTreeClassifier(max_features='sqrt', random_state=472432043), DecisionTreeClassifier(max_features='sqrt', random_state=1201263687), DecisionTreeC

## Weighted Parameter Aggregation 

In [16]:
global_model = RandomForestClassifier(random_state=42)

In [17]:
global_model.classes_ = np.array(attkLbl)
print(global_model.classes_)


['DoS' 'Infiltration' 'Port Scan' 'DDoS' 'Web Attack' 'BENIGN'
 'Heartbleed' 'Brute Force' 'Bot']


In [19]:
global_model.feature_names_in_ = np.array(features)
print(global_model.feature_names_in_)


['PC12' 'PC4' 'PC20' 'PC9' 'PC32' 'PC31' 'PC19' 'PC2' 'PC33' 'PC18' 'PC27'
 'PC21' 'PC1' 'PC24' 'PC34' 'PC8' 'PC16' 'PC30' 'PC17' 'PC29' 'PC5' 'PC23'
 'PC15' 'PC3' 'PC25' 'PC28' 'PC22' 'PC7' 'PC6' 'PC11' 'PC13' 'PC26' 'PC14'
 'PC10' 'PC35']


In [21]:
global_model.estimators_ = np.array(estimators)
print(global_model.estimators_)


[DecisionTreeClassifier(max_features='sqrt', random_state=1324556529)
 DecisionTreeClassifier(max_features='sqrt', random_state=1324556529)
 DecisionTreeClassifier(max_features='sqrt', random_state=423734972)
 DecisionTreeClassifier(max_features='sqrt', random_state=782698033)
 DecisionTreeClassifier(max_features='sqrt', random_state=2126768636)
 DecisionTreeClassifier(max_features='sqrt', random_state=249467210)
 DecisionTreeClassifier(max_features='sqrt', random_state=1972458954)
 DecisionTreeClassifier(max_features='sqrt', random_state=1111451555)
 DecisionTreeClassifier(max_features='sqrt', random_state=1572714583)
 DecisionTreeClassifier(max_features='sqrt', random_state=698027879)
 DecisionTreeClassifier(max_features='sqrt', random_state=1972458954)
 DecisionTreeClassifier(max_features='sqrt', random_state=249467210)
 DecisionTreeClassifier(max_features='sqrt', random_state=472432043)
 DecisionTreeClassifier(max_features='sqrt', random_state=1201263687)
 DecisionTreeClassifier(ma

## Aggregation

In [24]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import joblib

# Function to aggregate estimators using FedAvg
def fed_avg_aggregation(client_estimators_list):
    """
    Aggregates the estimators from multiple client models using FedAvg.
    """
    # Initialize an empty list to collect all trees from clients
    all_trees = []

    # Collect estimators from each client's model
    for estimators in client_estimators_list:
        all_trees.extend(estimators)

    # Compute average number of trees per model
    avg_tree_count = int(np.mean([len(estimators) for estimators in client_estimators_list]))

    # Select the first `avg_tree_count` estimators for the aggregated model
    aggregated_estimators = all_trees[:avg_tree_count]

    return aggregated_estimators

# Load client models from saved files
client_1_model = joblib.load('Sisira Horizontal/trial new/client_1_model.pkl')
client_2_model = joblib.load('Sisira Horizontal/trial new/client_2_model.pkl')
client_3_model = joblib.load('Sisira Horizontal/trial new/client_3_model.pkl')
client_4_model = joblib.load('Sisira Horizontal/trial new/client_4_model.pkl')
client_5_model = joblib.load('Sisira Horizontal/trial new/client_5_model.pkl')

# Example: Collecting the estimators from each model
client_models = [
    client_1_model.estimators_, 
    client_2_model.estimators_, 
    client_3_model.estimators_, 
    client_4_model.estimators_, 
    client_5_model.estimators_, 
]

# Perform aggregation of estimators using FedAvg
aggregated_estimators = fed_avg_aggregation(client_models)

# Initialize the global model
global_model = RandomForestClassifier(random_state=42)

# Assuming consistent feature names and classes across all clients
global_model.classes_ = np.array(attkLbl)  # Replace with the actual class labels if needed
print(global_model.classes_)
global_model.feature_names_in_ = np.array(features)  # Replace with actual feature names if needed
print(global_model.feature_names_in_)

# Set the aggregated estimators in the global model (this is where the aggregated "weights" are assigned)
global_model.estimators_ = np.array(aggregated_estimators)

# Print the aggregated estimators (decision trees) for the global model
print("Aggregated Estimators (Decision Trees):", global_model.estimators_)
print("Number of Aggregated Trees:", len(global_model.estimators_))

# The global model now contains the aggregated trees from all clients


['DoS' 'Infiltration' 'Port Scan' 'DDoS' 'Web Attack' 'BENIGN'
 'Heartbleed' 'Brute Force' 'Bot']
['PC12' 'PC4' 'PC20' 'PC9' 'PC32' 'PC31' 'PC19' 'PC2' 'PC33' 'PC18' 'PC27'
 'PC21' 'PC1' 'PC24' 'PC34' 'PC8' 'PC16' 'PC30' 'PC17' 'PC29' 'PC5' 'PC23'
 'PC15' 'PC3' 'PC25' 'PC28' 'PC22' 'PC7' 'PC6' 'PC11' 'PC13' 'PC26' 'PC14'
 'PC10' 'PC35']
Aggregated Estimators (Decision Trees): [DecisionTreeClassifier(max_features='sqrt', random_state=1608637542)
 DecisionTreeClassifier(max_features='sqrt', random_state=1273642419)
 DecisionTreeClassifier(max_features='sqrt', random_state=1935803228)
 DecisionTreeClassifier(max_features='sqrt', random_state=787846414)
 DecisionTreeClassifier(max_features='sqrt', random_state=996406378)
 DecisionTreeClassifier(max_features='sqrt', random_state=1201263687)
 DecisionTreeClassifier(max_features='sqrt', random_state=423734972)
 DecisionTreeClassifier(max_features='sqrt', random_state=415968276)
 DecisionTreeClassifier(max_features='sqrt', random_state=6700949

## Save Global Model

In [25]:
import joblib

# Save the global model as a .pkl file
global_model_filename = "global_model.pkl"
joblib.dump(global_model, global_model_filename)

print(f"Global model saved as '{global_model_filename}'")


Global model saved as 'global_model.pkl'


In [27]:

global_model_path = "Sisira Horizontal/trial new/global_model.pkl"
loaded_global_model = joblib.load(global_model_path)
print("Global model loaded successfully.")


Global model loaded successfully.


## Performance Evaluation


In [3]:
client3_data = pd.read_csv("Sisira Horizontal/trial new/Horizontally splitted data/client_3_data.csv")



In [7]:
from sklearn.model_selection import train_test_split

X = client3_data.drop(columns=["Attack Type"])
y = client3_data["Attack Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [87]:
pip install --upgrade scikit-learn


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 13.5 MB 3.1 MB/s            
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.2
    Uninstalling scikit-learn-1.4.2:
      Successfully uninstalled scikit-learn-1.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.26.0 requires scikit-learn<1.5.0,>=0.24, but you have scikit-learn 1.6.1 which is incompatible.
You should consider upgrading via the '/cm/local/apps/python39/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import joblib

# Load the global model
global_model = joblib.load('Sisira Horizontal/trial new/global_model.pkl')


In [13]:
print(global_model)


RandomForestClassifier(random_state=42)


In [18]:
# Paths for client models
client_model_paths = [
    "Sisira Horizontal/trial new/Fresh_models/client_1_model.pkl",
    "Sisira Horizontal/trial new/Fresh_models/client_2_model.pkl",
    "Sisira Horizontal/trial new/Fresh_models/client_3_model.pkl",
    "Sisira Horizontal/trial new/Fresh_models/client_4_model.pkl",
    "Sisira Horizontal/trial new/Fresh_models/client_5_model.pkl"
]

In [20]:
# Iterate through each client model file, load the model, and aggregate its trees
client_models = []
for client_model_path in client_model_paths:
    # Load the client's local model
    client_models.append(joblib.load(client_model_path))

len(client_models)

/jupyter/dtids.user01/.local/lib/python3.9/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/jupyter/dtids.user01/.local/lib/python3.9/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


5

In [21]:
attkLbl=[]
for model in client_models:
    thisModList = model.classes_.tolist()
    attkLbl.extend(thisModList)
    
attkLbl = list(set(attkLbl))
len(attkLbl)

9

In [69]:

features = []
for model in client_models:
    l = model.feature_names_in_.tolist()
    # print(l)
    features.extend(l)

features=list(set(features))
print(features)

['PC24', 'PC20', 'PC12', 'PC1', 'PC25', 'PC18', 'PC34', 'PC35', 'PC23', 'PC29', 'PC30', 'PC13', 'PC15', 'PC11', 'PC5', 'PC33', 'PC27', 'PC28', 'PC31', 'PC14', 'PC22', 'PC21', 'PC17', 'PC9', 'PC2', 'PC3', 'PC7', 'PC4', 'PC8', 'PC19', 'PC32', 'PC6', 'PC16', 'PC26', 'PC10']


In [30]:

estimators = []

for model in client_models:
    estimators.extend(model.estimators_)

estimators = list(set(estimators))
print("Estimators:", estimators)


Estimators: [DecisionTreeClassifier(max_features='sqrt', random_state=106328085), DecisionTreeClassifier(max_features='sqrt', random_state=61136438), DecisionTreeClassifier(max_features='sqrt', random_state=479546681), DecisionTreeClassifier(max_features='sqrt', random_state=479546681), DecisionTreeClassifier(max_features='sqrt', random_state=106328085), DecisionTreeClassifier(max_features='sqrt', random_state=1958805693), DecisionTreeClassifier(max_features='sqrt', random_state=2075460851), DecisionTreeClassifier(max_features='sqrt', random_state=2075460851), DecisionTreeClassifier(max_features='sqrt', random_state=1306710475), DecisionTreeClassifier(max_features='sqrt', random_state=389151677), DecisionTreeClassifier(max_features='sqrt', random_state=396917567), DecisionTreeClassifier(max_features='sqrt', random_state=1306710475), DecisionTreeClassifier(max_features='sqrt', random_state=504579232), DecisionTreeClassifier(max_features='sqrt', random_state=787716372), DecisionTreeClass

In [55]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Ensure that the test set features match the model's expected features in both order and number
X_test_imputed = X_test_imputed[global_model.feature_names_in_]




In [56]:
# Check for missing or extra columns in the test data
missing_columns = set(global_model.feature_names_in_) - set(X_test_imputed.columns)
extra_columns = set(X_test_imputed.columns) - set(global_model.feature_names_in_)

if missing_columns:
    print(f"Missing columns in test data: {missing_columns}")
if extra_columns:
    print(f"Extra columns in test data: {extra_columns}")



In [57]:
# Ensure there are no NaN values in the test data
nan_values = X_test_imputed.isna().sum()
if nan_values.any():
    print(f"NaN values found in test data: {nan_values[nan_values > 0]}")
    


In [58]:
# Ensure the order of features in the test set matches the training set
X_test_imputed = X_test_imputed[global_model.feature_names_in_]



In [60]:
print("Test features shape:", X_test_imputed.shape)
print("Model expected features:", global_model.feature_names_in_)


Test features shape: (100832, 35)
Model expected features: ['PC24' 'PC20' 'PC12' 'PC1' 'PC25' 'PC18' 'PC34' 'PC35' 'PC23' 'PC29'
 'PC30' 'PC13' 'PC15' 'PC11' 'PC5' 'PC33' 'PC27' 'PC28' 'PC31' 'PC14'
 'PC22' 'PC21' 'PC17' 'PC9' 'PC2' 'PC3' 'PC7' 'PC4' 'PC8' 'PC19' 'PC32'
 'PC6' 'PC16' 'PC26' 'PC10']


In [61]:
# Ensure X_test_imputed has the same columns as the model's expected feature names
X_test_imputed_aligned = X_test_imputed[global_model.feature_names_in_]

# Check the shape after alignment
print("Aligned test features shape:", X_test_imputed_aligned.shape)



Aligned test features shape: (100832, 35)


In [67]:
# Make predictions using the global model
y_pred_global = global_model.predict(X_test_imputed_aligned)
y_prob_global = global_model.predict_proba(X_test_imputed_aligned)

# Evaluate the model's performance using standard metrics
accuracy_global = accuracy_score(y_test, y_pred_global)
precision_global = precision_score(y_test, y_pred_global, average="weighted", zero_division=1)
recall_global = recall_score(y_test, y_pred_global, average="weighted", zero_division=1)
f1_global = f1_score(y_test, y_pred_global, average="weighted", zero_division=1)

# Print the evaluation results
global_metrics = {
    "Global Accuracy": accuracy_global * 100,
    "Global Precision (Weighted)": precision_global,
    "Global Recall (Weighted)": recall_global,
    "Global F1 Score (Weighted)": f1_global,
}

for metric, value in global_metrics.items():
    print(f"{metric}: {value}")

# If you want to show a detailed classification report for each class:
print("\nClassification Report:\n", classification_report(y_test, y_pred_global, target_names=global_model.classes_))

# Example of accessing the `estimators_` to view the individual models (e.g., decision trees in a random forest)
print("\nList of Estimators (Individual Models):")
for estimator in global_model.estimators_:
    print(estimator)

ValueError: operands could not be broadcast together with shapes (100832,9) (100832,8) (100832,9) 

In [82]:
# Check the classes in the model and the test data
print("Model classes:", global_model.classes_)
print("Unique classes in y_test:", np.unique(y_test))

# Ensure the number of classes match between the model and the test data
if len(global_model.classes_) != len(np.unique(y_test)):
    print("Warning: Class mismatch detected. Handling this case by using `predict` instead of `predict_proba`.")
    y_pred_global = global_model.predict(X_test_imputed_aligned)
    y_prob_global = None  # Set to None since `predict_proba` is not reliable in this case
else:
    # If the classes match, use `predict_proba`
    try:
        y_prob_global = global_model.predict_proba(X_test_imputed_aligned)
    except ValueError as e:
        print("Error in predict_proba:", e)
        y_prob_global = None  # Handle the case where predict_proba fails

    # Make predictions using the global model
    y_pred_global = global_model.predict(X_test_imputed_aligned)

# Evaluate the model's performance using standard metrics
accuracy_global = accuracy_score(y_test, y_pred_global)
precision_global = precision_score(y_test, y_pred_global, average="weighted", zero_division=1)
recall_global = recall_score(y_test, y_pred_global, average="weighted", zero_division=1)
f1_global = f1_score(y_test, y_pred_global, average="weighted", zero_division=1)

# Print the evaluation results
global_metrics = {
    "Global Accuracy": accuracy_global * 100,
    "Global Precision (Weighted)": precision_global,
    "Global Recall (Weighted)": recall_global,
    "Global F1 Score (Weighted)": f1_global,
}

for metric, value in global_metrics.items():
    print(f"{metric}: {value}")

# If you want to show a detailed classification report for each class:
print("\nClassification Report:\n", classification_report(y_test, y_pred_global, target_names=[str(i) for i in global_model.classes_]))

# Example of accessing the `estimators_` to view the individual models (e.g., decision trees in a random forest)
if hasattr(global_model, "estimators_"):
    print("\nList of Estimators (Individual Models):")
    for estimator in global_model.estimators_:
        print(estimator)
else:
    print("Model does not have estimators_ attribute.")



Model classes: ['DoS' 'DDoS' 'Bot' 'Web Attack' 'Port Scan' 'Heartbleed' 'BENIGN'
 'Brute Force' 'Infiltration']
Unique classes in y_test: ['BENIGN' 'Bot' 'Brute Force' 'DDoS' 'DoS' 'Heartbleed' 'Infiltration'
 'Port Scan' 'Web Attack']
Error in predict_proba: operands could not be broadcast together with shapes (100832,9) (100832,8) (100832,9) 


ValueError: operands could not be broadcast together with shapes (100832,9) (100832,8) (100832,9) 

In [70]:
print("Model expected classes:", global_model.classes_)


Model expected classes: ['DoS' 'DDoS' 'Bot' 'Web Attack' 'Port Scan' 'Heartbleed' 'BENIGN'
 'Brute Force' 'Infiltration']


In [71]:
# Align the test data with the features the model expects
X_test_imputed_aligned = X_test_imputed[global_model.feature_names_in_]
print("Aligned test features shape:", X_test_imputed_aligned.shape)


Aligned test features shape: (100832, 35)


In [72]:
print("Expected labels:", global_model.classes_)


Expected labels: ['DoS' 'DDoS' 'Bot' 'Web Attack' 'Port Scan' 'Heartbleed' 'BENIGN'
 'Brute Force' 'Infiltration']


In [75]:
y_pred_global = global_model.predict(X_test_imputed_aligned)


ValueError: operands could not be broadcast together with shapes (100832,9) (100832,8) (100832,9) 

In [76]:
print("Unique classes in test set:", np.unique(y_test))


Unique classes in test set: ['BENIGN' 'Bot' 'Brute Force' 'DDoS' 'DoS' 'Heartbleed' 'Infiltration'
 'Port Scan' 'Web Attack']


In [77]:
print("Test features columns:", X_test_imputed_aligned.columns)
print("Expected features:", global_model.feature_names_in_)


Test features columns: Index(['PC24', 'PC20', 'PC12', 'PC1', 'PC25', 'PC18', 'PC34', 'PC35', 'PC23',
       'PC29', 'PC30', 'PC13', 'PC15', 'PC11', 'PC5', 'PC33', 'PC27', 'PC28',
       'PC31', 'PC14', 'PC22', 'PC21', 'PC17', 'PC9', 'PC2', 'PC3', 'PC7',
       'PC4', 'PC8', 'PC19', 'PC32', 'PC6', 'PC16', 'PC26', 'PC10'],
      dtype='object')
Expected features: ['PC24' 'PC20' 'PC12' 'PC1' 'PC25' 'PC18' 'PC34' 'PC35' 'PC23' 'PC29'
 'PC30' 'PC13' 'PC15' 'PC11' 'PC5' 'PC33' 'PC27' 'PC28' 'PC31' 'PC14'
 'PC22' 'PC21' 'PC17' 'PC9' 'PC2' 'PC3' 'PC7' 'PC4' 'PC8' 'PC19' 'PC32'
 'PC6' 'PC16' 'PC26' 'PC10']


In [78]:
print("Class distribution in test set:", np.unique(y_test, return_counts=True))


Class distribution in test set: (array(['BENIGN', 'Bot', 'Brute Force', 'DDoS', 'DoS', 'Heartbleed',
       'Infiltration', 'Port Scan', 'Web Attack'], dtype=object), array([83668,    77,   395,  5136,  7819,     2,     1,  3652,    82]))


In [80]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Make predictions on the test set
y_pred_global = global_model.predict(X_test_imputed_aligned)




ValueError: operands could not be broadcast together with shapes (100832,9) (100832,8) (100832,9) 